# Multi-Turn Conversations with UnifiedLLM

This notebook demonstrates how to build conversational applications that maintain context across multiple turns.

**Topics covered:**
- Building a message history
- Creating a chat loop function
- Scripted conversations for testing
- Optional interactive chat
- Common mistakes to avoid

**Use cases:**
- Chatbots that remember conversation history
- Interactive tutoring systems
- Customer service assistants
- Code review assistants

## Setup

In [ ]:
from unifiedllm import LLM
from unifiedllm.errors import MissingAPIKeyError, ProviderAPIError
import os

In [ ]:
# Option: Set API key directly in notebook (if not already set via terminal)
# Uncomment and add your key if the export command didn't work:
# os.environ["GEMINI_API_KEY"] = "your-api-key-here"

# Check if API key is set
if not os.getenv("GEMINI_API_KEY"):
    print("⚠️  GEMINI_API_KEY environment variable is not set!")
    print("")
    print("Option 1 - Set via terminal:")
    print("  export GEMINI_API_KEY='your-api-key'")
    print("")
    print("Option 2 - Set in this notebook (uncomment line above):")
    print("  os.environ['GEMINI_API_KEY'] = 'your-api-key'")
else:
    print("✅ GEMINI_API_KEY is set!")

## Understanding Multi-Turn Conversations

In a multi-turn conversation:
1. Maintain a list of messages
2. Each message has a `role` ("user" or "model") and `content` (the text)
3. After each turn, append both the user's message and the model's response to the list
4. Pass the entire message history to maintain context

In [ ]:
# Initialize the LLM client
llm = LLM(
    provider="gemini",
    model="gemini-2.5-flash",
)

## Building a Chat Loop Function

Let's create a reusable function that handles multi-turn conversations.

In [ ]:
def chat_conversation(llm, user_messages, system_prompt=None):
    """
    Conduct a multi-turn conversation with the LLM.
    
    Args:
        llm: The LLM instance
        user_messages: List of user messages (strings)
        system_prompt: Optional system prompt to set behavior
    
    Returns:
        List of all messages (user + model)
    """
    # Set system prompt if provided
    if system_prompt:
        llm.system_prompt(system_prompt)
    
    # Initialize message history
    messages = []
    
    # Process each user message
    for user_msg in user_messages:
        print(f"\n👤 User: {user_msg}")
        
        # Add user message to history
        messages.append({"role": "user", "content": user_msg})
        
        # Get model response
        try:
            response = llm.chat(messages=messages)
            
            # Add model response to history
            messages.append({"role": "model", "content": response.text})
            
            print(f"🤖 Assistant: {response.text}")
            
        except Exception as e:
            print(f"❌ Error: {e}")
            break
    
    return messages

## Example 1: Scripted Conversation

Let's demonstrate a teaching scenario where context matters.

In [ ]:
# Define the conversation flow
user_inputs = [
    "I want to learn about Python functions.",
    "Can you show me a simple example?",
    "How do I add parameters to it?",
    "What's a return value?"
]

# Run the conversation
conversation = chat_conversation(
    llm,
    user_inputs,
    system_prompt="You are a patient Python tutor. Keep explanations concise and use code examples."
)

print(f"\n\n📝 Total messages in conversation: {len(conversation)}")

## Example 2: Problem-Solving Conversation

Let's see how the model maintains context while debugging.

In [ ]:
debugging_conversation = [
    "My Python code has an error: 'list index out of range'. What does this mean?",
    "Here's the code: numbers = [1, 2, 3]; print(numbers[5])",
    "How can I check the length of a list before accessing it?"
]

messages = chat_conversation(
    llm,
    debugging_conversation,
    system_prompt="You are a helpful debugging assistant. Provide clear, practical solutions."
)

## Viewing Message History

It's useful to inspect the complete message history.

In [ ]:
def print_messages(messages):
    """
    Pretty print the message history.
    """
    print("\n" + "="*60)
    print("CONVERSATION HISTORY")
    print("="*60)
    
    for i, msg in enumerate(messages, 1):
        role_emoji = "👤" if msg["role"] == "user" else "🤖"
        role_label = "User" if msg["role"] == "user" else "Assistant"
        
        print(f"\n{i}. {role_emoji} {role_label}:")
        print(f"   {msg['content'][:200]}...")  # Show first 200 chars
    
    print("\n" + "="*60)

# Print the conversation history
print_messages(messages)

## Interactive Chat (Optional)

**Note:** This cell uses `input()` for interactive chat. It won't run automatically.
Run it manually if you want to have a real-time conversation!

In [ ]:
# OPTIONAL: Interactive chat loop
# Uncomment and run this cell to chat interactively

def interactive_chat(llm, system_prompt=None):
    """
    Start an interactive chat session.
    Type 'quit' or 'exit' to end the conversation.
    """
    if system_prompt:
        llm.system_prompt(system_prompt)
    
    messages = []
    print("🤖 Chat started! Type 'quit' or 'exit' to end.\n")
    
    while True:
        user_input = input("👤 You: ")
        
        if user_input.lower() in ['quit', 'exit']:
            print("\n👋 Chat ended. Goodbye!")
            break
        
        messages.append({"role": "user", "content": user_input})
        
        try:
            response = llm.chat(messages=messages)
            messages.append({"role": "model", "content": response.text})
            print(f"🤖 Assistant: {response.text}\n")
        except Exception as e:
            print(f"❌ Error: {e}\n")
            break
    
    return messages

# Run the interactive chat
interactive_messages = interactive_chat(
    llm,
    system_prompt="You are a friendly AI assistant."
)

## Common Mistakes to Avoid

Here are some common pitfalls when building multi-turn conversations.

### Mistake 1: Invalid Role Names

Only "user" and "model" are valid roles. Other roles will raise a `ValueError`.

In [ ]:
# ❌ This will fail - invalid role
try:
    bad_messages = [
        {"role": "assistant", "content": "Hello"}  # Should be "model", not "assistant"
    ]
    response = llm.chat(messages=bad_messages)
except ValueError as e:
    print(f"❌ ValueError: {e}")

# ✅ Correct way
good_messages = [
    {"role": "user", "content": "Hello"}
]
response = llm.chat(messages=good_messages)
print(f"✅ Correct: {response.text[:50]}...")

### Mistake 2: Forgetting to Append Model Responses

If you don't add the model's responses to the message list, it will lose context.

In [ ]:
# ❌ Bad: Context is lost
messages = [{"role": "user", "content": "My name is Alice."}]
response1 = llm.chat(messages=messages)
print(f"Turn 1: {response1.text[:80]}")

# Not adding response1 to messages!
messages.append({"role": "user", "content": "What's my name?"})  # Model won't remember
response2 = llm.chat(messages=messages)
print(f"\nTurn 2 (without context): {response2.text[:80]}")

print("\n" + "="*60 + "\n")

# ✅ Good: Context is maintained
messages = [{"role": "user", "content": "My name is Bob."}]
response1 = llm.chat(messages=messages)
print(f"Turn 1: {response1.text[:80]}")

# Add model response before next turn
messages.append({"role": "model", "content": response1.text})
messages.append({"role": "user", "content": "What's my name?"})
response2 = llm.chat(messages=messages)
print(f"\nTurn 2 (with context): {response2.text[:80]}")

### Mistake 3: Missing API Key

Always ensure your API key is set before making requests.

In [ ]:
# This pattern checks for the key before making any requests
def safe_chat(llm, messages):
    try:
        response = llm.chat(messages=messages)
        return response
    except MissingAPIKeyError:
        print("❌ API key is missing! Please set GOOGLE_API_KEY environment variable.")
        return None
    except ProviderAPIError as e:
        print(f"❌ API Error: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

# Example usage
test_messages = [{"role": "user", "content": "Hello!"}]
result = safe_chat(llm, test_messages)
if result:
    print(f"✅ Success: {result.text[:50]}...")

## Best Practices Summary

✅ **Do:**
- Always use "user" and "model" as roles
- Append both user messages AND model responses to maintain context
- Set a system prompt to guide the assistant's behavior
- Handle errors gracefully with try/except blocks
- Keep track of message history for debugging

❌ **Don't:**
- Use invalid role names like "assistant" or "system"
- Forget to append model responses to the message list
- Make API calls without checking for errors
- Pass both `prompt` and `messages` parameters together (use one or the other)

**Next steps:**
- Try building a simple chatbot for your specific use case
- Experiment with different system prompts to change behavior
- Check out `provider_comparison.ipynb` to see how different providers handle conversations